In [13]:
from scipy.io import loadmat
import cv2
import shutil
import pandas as pd
import os

images_dir = "../car_ims_2/car_ims_2/car_ims/"
resp = loadmat("../cars_annos.mat")

### Create list of class names

In [14]:
classes = []
for class_name in resp.get("class_names")[0]:
    classes.append(class_name[0])

### Create CSV File

We will create a csv file for both train and test dataset and also store images in directories accordingly

In [19]:
def get_data(test_type, directory):
    data = []
    if not os.path.exists(directory): os.mkdir(directory)
    for annot in resp.get("annotations")[0]:
        test = int(annot[6][0][0])
        if test == test_type: continue


        image_name = annot[0][0].split("/")[-1]
        image = cv2.imread(images_dir + image_name)
        shutil.copy(images_dir + image_name, f"{directory}/" + image_name)
        h, w, _ = image.shape
        xmin = int(annot[1][0][0])
        ymin = int(annot[2][0][0])
        xmax = int(annot[3][0][0])
        ymax = int(annot[4][0][0])
        class_name = classes[annot[5][0][0] - 1]

        data.append({
            'filename' : image_name, "width" : w, "height" : h, 
            'class' : class_name,
            'xmin' : xmin, 'ymin' : ymin, 'xmax' : xmax, 
            'ymax' : ymax})
        
    df = pd.DataFrame(data)
    return df

### Save data
save labels for both train and test dataset and also save labels txt file

In [20]:
# Get train data
df = get_data(0, "train")
df.to_csv("train_labels.csv", index=None)

In [21]:
# Get train data
df = get_data(1, "test")
df.to_csv("test_labels.csv", index=None)

In [22]:
with open("labels.pbtxt", "w") as file:
    for index, cls in enumerate(classes):
        row = "item {\n\tid: " + str(index + 1) + "\n\tname: '" + cls +"'\n}\n"
        file.write(row)

In [26]:
import json
labels = {cls:i + 1 for i, cls in enumerate(classes)}
with open('labels.json', "w") as file:
    json.dump(labels, file)